In [ ]:
from google.colab import drive

drive.mount('/content/drive')
root_dir = "/content/drive/My Drive/"
base_dir = root_dir + 'Tesi/'

In [ ]:
import pandas as pd

In [ ]:
from itertools import chain
from collections import Counter

In [ ]:
df = pd.read_csv(base_dir + 'downstream/ccoha2_preprocessed.csv')
df.shape

GENSIM PHRASES

In [ ]:
import re  # For preprocessing
import pandas as pd  # For data handling
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = stopwords.words('english')

In [ ]:
df['gensimtext'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
punct = ''''"“”’'.,<>!\()\\|[]{}~;:?`+=@#$%^&*'''

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in punct])
    return text

In [ ]:
df['gensimtext'] = df['gensimtext'].apply(lambda x: remove_punct(x)) 

In [ ]:
df['gensimtext']

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
sent = [row.split() for row in df['gensimtext']]

In [ ]:
bigram = Phrases(sent, min_count=5, threshold=100) #3

trigram  = Phrases(bigram[sent], min_count=5, threshold=100) #3

fourgram  = Phrases(trigram[sent], min_count=5, threshold=100) #5

In [ ]:
bigram_phraser = Phraser(bigram)

In [ ]:
trigram_phraser = Phraser(trigram)

In [ ]:
fourgram_phraser = Phraser(fourgram)

In [ ]:
sentences = bigram_phraser[sent]

In [ ]:
sentences = trigram_phraser[sentences]

In [ ]:
sentences = fourgram_phraser[sentences]

In [ ]:
wordphrases = []

for w in sentences:
  wordphrases.append(w)

In [ ]:
for i in range(len(wordphrases)):
  wordphrases[i]= ' '.join(wordphrases[i])

In [ ]:
dfGensim = pd.DataFrame(data=wordphrases, columns=["text"])

In [ ]:
for i in range(len(dfGensim)):
  dfGensim['text'][i]= dfGensim['text'][i].split()

In [ ]:
dfGensim['text']

In [ ]:
gensim_list = []

In [ ]:
df['keywords'] = ""

In [ ]:
for i in range(len(dfGensim)):
  for el in dfGensim['text'][i]:
    if '_' in el:
      el = el.replace("_", " ")
      el = el.lower()
      gensim_list.append(el)
  df['keywords'][i]=gensim_list
  gensim_list = []

In [ ]:
df = df.drop(columns=['gensimtext'])

MWE THRESHOLD

In [ ]:
mwe_counter = Counter(chain.from_iterable(df['keywords'])) 

In [ ]:
df['keywords'] = [([j for j in i if mwe_counter[j] >= 5]) for i in df['keywords']]

MWE SUBSTITUTION

In [ ]:
for i in range(len(df)):
  df['text'][i] = re.sub('('+'|'.join('\\b'+re.escape(g)+'\\b' for g in df['keywords'][i])+')', lambda m: m.group(0).replace(' ', '_'), df['text'][i], flags=re.I)

In [ ]:
df

SAVE

In [ ]:
compression_opts = dict(method='zip', archive_name='newCCOHA2_Gensim_5-100s.csv')

df['text'].to_csv('newCCOHA2_Gensim_5-100s.zip', index=False, compression=compression_opts)

In [ ]:
compression_opts = dict(method='zip', archive_name='preprocessed_Gensim.csv')

df.to_csv('preprocessed_Gensim.zip', index=False, compression=compression_opts)